## Alert Manager

Zur Prometheus Instanz erstellen wir eine Alert Manager Instanz

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: Alertmanager
metadata:
  name: alertmanager
spec:
  replicas: 1
  resources:
    requests:
      memory: 200Mi
---
apiVersion: v1
kind: Service
metadata:
  name: alertmanager
spec:
  ports:
    - name: web
      port: 9093
      targetPort: 9093
  type: NodePort      
  selector:
    alertmanager: alertmanager        
%EOF%


## Alert Manager UI

In [ ]:
%%bash
echo "Alert Manager UI: http://$(cat ~/work/server-ip):$(kubectl get service --namespace ms-rest alertmanager -o=jsonpath='{ .spec.ports[0].nodePort }')"

### Alert Rules 

Damit der Alert Manager etwas zu tun bekommt, übergeben wir diesem ein paar Aufgaben:

**1. HTTP GET-Rate ist ungewöhnlich hoch**

Diese Regel löst aus, wenn die Anzahl der HTTP GET-Anfragen pro Sekunde über einem Schwellenwert liegt.
(Anpassbar je nach Systemlast)

Was macht diese Regel?
* Berechnet die Rate der HTTP GET-Anfragen (rate(http_requests_total{method="GET"}[5m])).
* Löst aus, wenn der Wert über 100 pro Sekunde liegt.
* for: 2m → Erst wenn das 2 Minuten lang anhält.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  name: http-traffic
  namespace: ms-rest
spec:
  groups:
    - name: http.rules
      rules:
        - alert: HighHttpGetRate
          expr: rate(http_requests_total{method="GET"}[5m]) > 100
          for: 2m
          labels:
            severity: warning
          annotations:
            summary: "Hohe HTTP GET-Rate erkannt"
            description: "Die Anzahl der HTTP GET-Anfragen liegt über 100 pro Sekunde in den letzten 5 Minuten."
%EOF%


**2. Speicherverbrauch pro Pod zu hoch**

Diese Regel löst aus, wenn ein einzelner Pod mehr als 500 MB RAM verwendet.

Was macht diese Regel?
* Prüft, ob ein Pod mehr als 500 MB RAM nutzt (process_memory_usage_bytes > 500 * 1024 * 1024).
* Löst aus, wenn das 2 Minuten lang anhält.


In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace ms-rest -f -
apiVersion: monitoring.coreos.com/v1
kind: PrometheusRule
metadata:
  name: memory-overflow
  namespace: ms-rest
spec:
  groups:
    - name: memory.rules
      rules:
        - alert: HighMemoryUsagePerPod
          expr: process_memory_usage_bytes > 500 * 1024 * 1024
          for: 2m
          labels:
            severity: critical
          annotations:
            summary: "Hoher Speicherverbrauch erkannt"
            description: "Ein Pod verbraucht mehr als 500 MB RAM."
%EOF%

Ändert die 500 auf 5 in der memory.rules und beobachtet die Regeln in Prometheus und dem Alert-Manager

---

### Aufräumen

In [ ]:
%%bash
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/catalog-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/customer-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/order-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-0-0-deployment/webshop-deployment.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/catalog-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/customer-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/order-service.yaml
kubectl delete --namespace ms-rest -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/webshop-service.yaml
kubectl delete ns ms-rest